## 데이터 출처 : https://grouplens.org/datasets/movielens/25m/

In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("sparkml_movie_recommendation")\
    .config("spark.executor.memory",MAX_MEMORY)\
    .config("spark.driver.memory",MAX_MEMORY)\
    .getOrCreate()

22/05/23 15:56:13 WARN Utils: Your hostname, ihyeonmin-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.11 instead (on interface en0)
22/05/23 15:56:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/23 15:56:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/23 15:56:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/23 15:56:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
ratings_file = "/Users/ihyeonmin/Desktop/study/data-engineering/01-spark/data/ml-25m/ratings.csv"
ratings_df = spark.read.csv(f"file:///{ratings_file}",inferSchema=True,header=True)

In [4]:
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [5]:
ratings_df = ratings_df.select(["userId","movieId","rating"])

In [6]:
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [7]:
ratings_df.select("rating").describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423508|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [8]:
train_df,test_df = ratings_df.randomSplit([0.8,0.2])

In [9]:
from pyspark.ml.recommendation import ALS

In [10]:
als = ALS(
    maxIter=5,
    regParam=0.1,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)

In [11]:
model=als.fit(train_df)

22/05/23 15:57:09 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/05/23 15:57:09 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/05/23 15:57:10 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
predictions = model.transform(test_df)

In [13]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    76|   1342|   3.5|  3.031944|
|   296|   1580|   3.0| 2.6825006|
|   321|   3175|   3.0| 3.2240853|
|   368|  54190|   4.0|  3.451634|
|   501|   1580|   5.0| 3.9116085|
|   501|   1645|   4.0| 3.6053133|
|   588|   1645|   2.5|  2.634791|
|   596|   1580|   3.0| 3.5761979|
|   597|   1591|   2.0| 2.5032573|
|   606|   1580|   5.0|  4.222599|
|   606|   1645|   3.0| 4.1143756|
|   606|   4519|   4.0|  3.782704|
|   606|  68135|   3.5| 4.0179014|
|   606| 160563|   4.0| 3.9945238|
|   613|   1088|   4.0| 3.1805325|
|   626|   2122|   2.0| 2.2775686|
|   626|   2366|   3.0| 3.1631243|
|   633|   1591|   5.0| 3.1724758|
|   772|    471|   4.0| 3.5199957|
|   804|   3175|   5.0| 4.4561524|
+------+-------+------+----------+
only showing top 20 rows



In [15]:
predictions.select("rating","prediction").describe().show()

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           4997956|           4997956|
|   mean|3.5343081251615662|3.4194287718659613|
| stddev|1.0608288528728944|0.6424643642115752|
|    min|               0.5|        -1.1333814|
|    max|               5.0|         6.7938986|
+-------+------------------+------------------+



In [18]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

In [19]:
rmse = evaluator.evaluate(predictions)

In [20]:
print(rmse)

0.8106739576782812


In [21]:
model.recommendForAllUsers(3).show()

/usr/local/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{203086, 5.93246...|
|    27|[{203086, 6.27643...|
|    28|[{194434, 7.40544...|
|    31|[{194334, 3.82996...|
|    34|[{194434, 5.35815...|
|    44|[{199187, 6.73015...|
|    53|[{162436, 7.34231...|
|    65|[{194434, 6.31207...|
|    76|[{203086, 6.32695...|
|    78|[{176729, 6.52199...|
|    81|[{138580, 5.13599...|
|    85|[{203086, 5.99154...|
|   101|[{203086, 5.29009...|
|   103|[{183947, 5.71310...|
|   108|[{203086, 5.35401...|
|   115|[{203086, 5.99874...|
|   126|[{203086, 6.55799...|
|   133|[{203086, 5.31141...|
|   137|[{203086, 5.78383...|
|   148|[{183947, 5.76353...|
+------+--------------------+
only showing top 20 rows




[Stage 229:>                                                        (0 + 1) / 1]



In [22]:
model.recommendForAllItems(3).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     12|[{87426, 5.213289...|
|     26|[{105801, 5.13789...|
|     27|[{87426, 5.402856...|
|     28|[{105801, 5.43870...|
|     31|[{87426, 5.215098...|
|     34|[{128562, 5.50455...|
|     44|[{87426, 5.173346...|
|     53|[{27587, 5.178305...|
|     65|[{87426, 5.271449...|
|     76|[{87426, 5.031163...|
|     78|[{105801, 4.71758...|
|     81|[{156318, 4.90050...|
|     85|[{54622, 4.984208...|
|    101|[{105801, 5.01496...|
|    103|[{26659, 5.147465...|
|    108|[{100653, 5.13729...|
|    115|[{105801, 5.78943...|
|    126|[{87426, 4.734802...|
|    133|[{100653, 4.22847...|
|    137|[{100653, 4.94458...|
+-------+--------------------+
only showing top 20 rows



In [24]:
from pyspark.sql.types import IntegerType

user_list = [65,78,81]
users_df = spark.createDataFrame(user_list,IntegerType()).toDF('userID')

users_df.show()


[Stage 259:>                                                        (0 + 1) / 1]



+------+
|userID|
+------+
|    65|
|    78|
|    81|
+------+



In [26]:
user_recs = model.recommendForUserSubset(users_df, 5)

In [27]:
movies_list = user_recs.collect()[0].recommendations

In [29]:
recs_df=spark.createDataFrame(movies_list)
recs_df.show()

+-------+-----------------+
|movieId|           rating|
+-------+-----------------+
| 194434|6.312077522277832|
| 185645|6.165622711181641|
| 169606| 6.09890604019165|
| 192261|5.982861518859863|
| 126941|5.934837818145752|
+-------+-----------------+



In [30]:
movies_file = "/Users/ihyeonmin/Desktop/study/data-engineering/01-spark/data/ml-25m/movies.csv"
movies_df = spark.read.csv(f"file:///{movies_file}",inferSchema=True,header=True)


[Stage 362:>                                                        (0 + 1) / 1]



In [31]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [32]:
recs_df.createOrReplaceTempView("recommendations")
movies_df.createOrReplaceTempView("movies")

In [35]:
query="""
SELECT
    *
FROM
    movies JOIN recommendations
    ON movies.movieId = recommendations.movieId
ORDER BY
    rating desc
"""

recommended_movies = spark.sql(query)
recommended_movies.show()

+-------+--------------------+------------------+-------+-----------------+
|movieId|               title|            genres|movieId|           rating|
+-------+--------------------+------------------+-------+-----------------+
| 194434|   Adrenaline (1990)|(no genres listed)| 194434|6.312077522277832|
| 185645|Stone Cold Steve ...|       Documentary| 185645|6.165622711181641|
| 169606|Dara O'Briain Cro...|            Comedy| 169606| 6.09890604019165|
| 192261|Don't Laugh at My...|      Comedy|Drama| 192261|5.982861518859863|
| 126941|Joni's Promise (2...|            Comedy| 126941|5.934837818145752|
+-------+--------------------+------------------+-------+-----------------+



In [36]:
def get_recommendations(user_id,num_recs):
    users_df = spark.createDataFrame([user_id],IntegerType()).toDF('userId')
    user_recs_df = model.recommendForUserSubset(users_df,num_recs)
    
    recs_list = user_recs_df.collect()[0].recommendations
    recs_df = spark.createDataFrame(recs_list)
    recommended_movies = spark.sql(query)
    return recommended_movies

In [37]:
recs = get_recommendations(456,10)

In [38]:
recs.toPandas()

,movieId,title,genres,movieId,rating
0,194434,Adrenaline (1990),(no genres listed),194434,6.312078
1,185645,Stone Cold Steve Austin: The Bottom Line on th...,Documentary,185645,6.165623
2,169606,Dara O'Briain Crowd Tickler (2015),Comedy,169606,6.098906
3,192261,Don't Laugh at My Romance (2008),Comedy|Drama,192261,5.982862
4,126941,Joni's Promise (2005),Comedy,126941,5.934838


In [39]:
spark.stop()